In [11]:
#score: 0.96397
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools
import pandas as pd

In [6]:
# import mnist data and visualize first image
X_train = np.load("/kaggle/input/audio-binary-classification/train_data.npy")
X_test = np.load("/kaggle/input/audio-binary-classification/test_data.npy")
y_train = pd.read_csv('/kaggle/input/audio-binary-classification/train_labels.csv').loc[:,'Label'].values

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train = np.reshape(X_train, (-1, 210,210,1))
X_test = np.reshape(X_test, (X_test.shape[0], 210,210,1))


In [12]:
print('Build LSTM RNN model ...')
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2],1)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(units=64, dropout=0.05, recurrent_dropout=0.20, return_sequences=True))
model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.20, return_sequences=False))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

model.summary()

Build LSTM RNN model ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 208, 208, 64)      640       
_________________________________________________________________
dropout_3 (Dropout)          (None, 208, 208, 64)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 206, 206, 32)      18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 103, 103, 32)      0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 103, 3296)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 103, 64)           860416    
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)            

In [13]:

# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 100, epochs = 50)

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = model.predict(X_test)
#np.savetxt("y_pred_ann.csv", y_pred, delimiter=",")
  
df2 = pd.DataFrame(y_pred, index = range(y_pred.shape[0]))
df2.reset_index(level=0, inplace=True)
df2.columns=['Id', 'Label']
df2.to_csv("y_pred_conv_LSTM_64_32.csv", index=False)


Epoch 1/50
3999/3999 [==============================] - 29s 7ms/step - loss: 0.2723 - acc: 0.8782
Epoch 2/50
3999/3999 [==============================] - 22s 6ms/step - loss: 0.2402 - acc: 0.8780
Epoch 3/50
3999/3999 [==============================] - 23s 6ms/step - loss: 0.2307 - acc: 0.8785
Epoch 4/50
3999/3999 [==============================] - 22s 5ms/step - loss: 0.2179 - acc: 0.8950
Epoch 5/50
3999/3999 [==============================] - 22s 5ms/step - loss: 0.2053 - acc: 0.8972
Epoch 6/50
3999/3999 [==============================] - 22s 5ms/step - loss: 0.1990 - acc: 0.9010
Epoch 7/50
3999/3999 [==============================] - 22s 5ms/step - loss: 0.1899 - acc: 0.9100
Epoch 8/50
3999/3999 [==============================] - 22s 5ms/step - loss: 0.1966 - acc: 0.8985
Epoch 9/50
3999/3999 [==============================] - 22s 5ms/step - loss: 0.1850 - acc: 0.9082
Epoch 10/50
3999/3999 [==============================] - 21s 5ms/step - loss: 0.1899 - acc: 0.9115
Epoch 11/50
3999/39